In [6]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [7]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [17]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI environment. env.P represents the transition probabilities of the environment.
        theta: Stopping threshold. If the value of all states changes less than theta
            in one iteration we are done.
        discount_factor: lambda time discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function. 
    思路：
        对于每个state，获取它的所有行动的行动值函数，将这个行动值函数作为当前状态的状态值函数。
        直到更新值小到一定程度，结束。
        再根据结果进行查找路径，得到policy。
    """
    

    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])
    while True:
        delta = 0
        for s in range(env.nS):
            Action_values = np.zeros(env.nA)
            for action in range(env.nA):
                for prob, next_state, reward, done in env.P[s][action]:
                    Action_values[action] = prob * (reward + V[next_state] * discount_factor)
            Action_value_max = np.max(Action_values)
            delta = max(delta, np.abs(Action_value_max - V[s]))
            V[s] = Action_value_max
        if delta < theta:
            break
            
    for s in range(env.nS):
        Action_values = np.zeros(env.nA)
        for action in range(env.nA):
            for prob, next_state, reward, done in env.P[s][action]:
                Action_values[action] = prob * (reward + V[next_state] * discount_factor)
        Action_max = np.argmax(Action_values)
        policy[s][Action_max] = 1
    
    return policy, V

In [18]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [19]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)